In [ ]:
import xarray as xr
import hvplot.xarray
import panel as pn
import pandas as pd
from holoviews.element import tiles
import holoviews as hv
import geoviews as gv
import cartopy.crs as ccrs
import fsspec
import numpy as np
import glob
import dask
from dask.distributed import Client, LocalCluster
from dask.diagnostics import ProgressBar
gv.extension('bokeh',logo=False)
hv.extension('bokeh',logo=False)
pn.extension()

In [ ]:
pd.__version__

In [ ]:
get_tile_id = False

if get_tile_id:
    df = pd.read_csv('https://hls.gsfc.nasa.gov/wp-content/uploads/2016/10/S2_TilingSystem2-1.txt',delim_whitespace=True)
    cper_lat = 40.8406
    cper_lon = -104.71538
    print(df[(df.MinLon<=cper_lon)&(df.MaxLon>=cper_lon)&(df['MinLon.1']<=cper_lat)&(df['MaxLon.1']>=cper_lat)])

In [ ]:
#extents of CPER
min_x = 517617.218700
max_x = 527253.409100
min_y = 4514729.500000
max_y = 4524372.500000

In [ ]:
fs = fsspec.filesystem('https')
S30_yrs = np.arange(2015,2020,dtype=int)
L30_yrs = np.arange(2013,2020,dtype=int)

# Get list of all Files
f_names = {'Sentinel':[],
           'Landsat':[]}
S_prefix = 'https://hlssa.blob.core.windows.net/hls/S309/'
L_prefix = 'https://hlssa.blob.core.windows.net/hls/L309/'
for yr in S30_yrs:
    for fi in fs.glob('https://hls.gsfc.nasa.gov/data/v1.4/S30/'+str(yr)+'/13/T/E/F/*.hdf'):
        bname = glob.os.path.basename(fi)[0:-4]
        f_names['Sentinel'].append(S_prefix+bname)
for yr in L30_yrs:
    for fi in fs.glob('https://hls.gsfc.nasa.gov/data/v1.4/L30/'+str(yr)+'/13/T/E/F/*.hdf'):
        bname = glob.os.path.basename(fi)[0:-4]
        f_names['Landsat'].append(L_prefix+bname)
        
cluster = LocalCluster(threads_per_worker=1)
client = Client(cluster)
client

In [ ]:
#Read Data from the Azure Blob Storage
def read_dat(blob_file):
    try:
        if 'S30' in blob_file:
            red = xr.open_rasterio(blob_file+'_04.tif',chunks={}).sel(y=slice(max_y,min_y),x=slice(min_x,max_x))
            nir = xr.open_rasterio(blob_file+'_09.tif',chunks={}).sel(y=slice(max_y,min_y),x=slice(min_x,max_x))
        if 'L30' in blob_file:
            red = xr.open_rasterio(blob_file+'_04.tif',chunks={}).sel(y=slice(max_y,min_y),x=slice(min_x,max_x))
            nir = xr.open_rasterio(blob_file+'_05.tif',chunks={}).sel(y=slice(max_y,min_y),x=slice(min_x,max_x))
        ndvi = (nir - red) / (nir + red)
        ndvi = ndvi.rename(band='date')
        ndvi = ndvi.assign_coords(date = [pd.to_datetime(blob_file[60:-5],format='%Y%j')])
        ndvi = ndvi.persist()
        return(ndvi)
    except:
        return()

In [ ]:
#Download Data and combine into a single xarray object
ndvi = []
for files in f_names['Sentinel'][0:3]:
    ndvi.append(dask.delayed(read_dat)(files))
for files in f_names['Landsat'][0:3]:
    ndvi.append(dask.delayed(read_dat)(files))

ndvi = dask.compute(*ndvi)
ndvi = xr.combine_by_coords([n.to_dataset(name='ndvi') for n in ndvi if n is not ()])

In [ ]:
#Show
pp = pn.Row(ndvi.hvplot.image(x='x',
                              y='y',
                              crs=32613,
                              cmap='viridis',
                              min_width=1000,
                              min_height=800).opts(alpha=.7)*gv.tile_sources.EsriImagery())
pp.servable()